# Group Exercises

In [207]:
import plotly.express as px
import pandas as pd
import duckdb as db

df = pd.read_csv("data/influensa.csv")

df

,Datum,Vårdform,Mått,DATA
0,2015-07-09,Slutenvård,Antal,0.0
1,2015-07-09,Slutenvård,Per 100 000,0.000
2,2015-07-09,Intensivvård,Antal,0.0
3,2015-07-09,Intensivvård,Per 100 000,0.000
4,2015-07-16,Slutenvård,Antal,0.0
...,...,...,...,...
2035,2025-04-03,Intensivvård,Per 100 000,0.016
2036,2025-04-10,Slutenvård,Antal,.
2037,2025-04-10,Slutenvård,Per 100 000,.
2038,2025-04-10,Intensivvård,Antal,1.1


In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Datum     2040 non-null   object
 1   Vårdform  2040 non-null   object
 2   Mått      2040 non-null   object
 3   DATA      2040 non-null   object
dtypes: object(4)
memory usage: 63.9+ KB


In [209]:
df["DATA"] = df["DATA"].str.replace(".", "0").astype(float)

In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2040 entries, 0 to 2039
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Datum     2040 non-null   object 
 1   Vårdform  2040 non-null   object 
 2   Mått      2040 non-null   object 
 3   DATA      2040 non-null   float64
dtypes: float64(1), object(3)
memory usage: 63.9+ KB


In [211]:
df['Datum'] = pd.to_datetime(df['Datum'])
  

In [212]:
df["År"] = df["Datum"].dt.year
df["Månad"] = df["Datum"].dt.month


In [213]:
df.head()

,Datum,Vårdform,Mått,DATA,År,Månad
0,2015-07-09,Slutenvård,Antal,0.0,2015,7
1,2015-07-09,Slutenvård,Per 100 000,0.0,2015,7
2,2015-07-09,Intensivvård,Antal,0.0,2015,7
3,2015-07-09,Intensivvård,Per 100 000,0.0,2015,7
4,2015-07-16,Slutenvård,Antal,0.0,2015,7


In [214]:
df["Mått"]

0             Antal
1       Per 100 000
2             Antal
3       Per 100 000
4             Antal
           ...     
2035    Per 100 000
2036          Antal
2037    Per 100 000
2038          Antal
2039    Per 100 000
Name: Mått, Length: 2040, dtype: object

In [215]:
df_vård = db.query("""--sql
         SELECT År, Månad , Vårdform ,Data, Mått
         FROM df
         WHERE (Mått == 'Antal')
         """).df()

df_vård

,År,Månad,Vårdform,DATA,Mått
0,2015,7,Slutenvård,0.0,Antal
1,2015,7,Intensivvård,0.0,Antal
2,2015,7,Slutenvård,0.0,Antal
3,2015,7,Intensivvård,0.0,Antal
4,2015,7,Slutenvård,6.0,Antal
...,...,...,...,...,...
1015,2025,3,Intensivvård,206.0,Antal
1016,2025,4,Slutenvård,0.0,Antal
1017,2025,4,Intensivvård,107.0,Antal
1018,2025,4,Slutenvård,0.0,Antal


In [217]:
df_vård = db.query("""--sql
         SELECT År || '-' || LPAD(CAST(Månad AS VARCHAR),2,'0') AS Datum, Vårdform ,Data, Mått
         FROM df
         WHERE (Mått == 'Antal')
         """).df()

df_vård

,Datum,Vårdform,DATA,Mått
0,2015-07,Slutenvård,0.0,Antal
1,2015-07,Intensivvård,0.0,Antal
2,2015-07,Slutenvård,0.0,Antal
3,2015-07,Intensivvård,0.0,Antal
4,2015-07,Slutenvård,6.0,Antal
...,...,...,...,...
1015,2025-03,Intensivvård,206.0,Antal
1016,2025-04,Slutenvård,0.0,Antal
1017,2025-04,Intensivvård,107.0,Antal
1018,2025-04,Slutenvård,0.0,Antal


In [218]:
df_vård.head()

,Datum,Vårdform,DATA,Mått
0,2015-07,Slutenvård,0.0,Antal
1,2015-07,Intensivvård,0.0,Antal
2,2015-07,Slutenvård,0.0,Antal
3,2015-07,Intensivvård,0.0,Antal
4,2015-07,Slutenvård,6.0,Antal


In [219]:
df_summerad = (df_vård.groupby(["Datum", "Vårdform"], as_index=False).agg({"DATA": "sum"}))

In [220]:
df_summerad

,Datum,Vårdform,DATA
0,2015-07,Intensivvård,0.0
1,2015-07,Slutenvård,12.0
2,2015-08,Intensivvård,0.0
3,2015-08,Slutenvård,15.0
4,2015-09,Intensivvård,0.0
...,...,...,...
231,2025-02,Slutenvård,41223.0
232,2025-03,Intensivvård,1117.0
233,2025-03,Slutenvård,31325.0
234,2025-04,Intensivvård,208.0


In [221]:
fig = px.line(df_vård, x="Datum", y="DATA",markers=False,color="Vårdform")

fig.update_traces(line=dict(width= 3))
fig.update_layout(plot_bgcolor="white")

fig.show()

In [222]:
fig = px.line(df_summerad, x="Datum", y="DATA",markers=True,color="Vårdform")

fig.update_traces(line=dict(width= 2))
fig.update_layout(plot_bgcolor="white", hovermode="x unified")

fig.show()

In [ ]:
df_vård = db.query("""--sql
         SELECT År || '-' || LPAD(CAST(Månad AS VARCHAR),2,'0') AS Datum, Vårdform ,Data, Mått
         FROM df
         WHERE (Mått == 'Antal')
         """).df()

df_vård

EXTRACT(YEAR FROM timestamp_column) AS year

In [225]:
fig = px.bar(df_summerad, x="Datum", y="DATA",color="Vårdform")

fig.update_layout(plot_bgcolor="white", hovermode="x unified")

fig.show()